In [16]:
import random

def knapsack_genetic(v, w, K, N=50, s=0.5, c=0.8, m=0.1, maxIter=50):
    num_items = len(v)
    
    # Generar la población inicial de individuos aleatorios
    def generate_individual():
        return [random.randint(0, 1) for _ in range(num_items)]
    
    def generate_population(size):
        return [generate_individual() for _ in range(size)]

    # Función de fitness que evalúa el valor total y penaliza el exceso de peso
    def fitness(individual):
        total_value = sum(v[i] * individual[i] for i in range(num_items))
        total_weight = sum(w[i] * individual[i] for i in range(num_items))
        if total_weight > K:
            return 0  # Penalización
        return total_value

    # Selección por torneo
    def selection(population, retain_rate):
        graded = [(fitness(individual), individual) for individual in population]
        graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]
        retain_length = int(len(graded) * retain_rate)
        return graded[:retain_length]

    # Cruce de un punto
    def crossover(parent1, parent2):
        point = random.randint(1, num_items - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

    # Mutación con probabilidad m
    def mutate(individual, mutation_rate):
        for i in range(num_items):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i]  # Cambia 0 a 1 o 1 a 0
        return individual

    # Algoritmo principal
    population = generate_population(N)
    best_solution = None
    best_value = 0
    
    for _ in range(maxIter):
        # Evaluar la población actual
        population = sorted(population, key=lambda ind: fitness(ind), reverse=True)
        if fitness(population[0]) > best_value:
            best_solution = population[0]
            best_value = fitness(population[0])

        # Selección
        selected = selection(population, s)
        
        # Cruce para generar nuevos individuos
        children = []
        while len(children) < int(N * c):
            parent1 = random.choice(selected)
            parent2 = random.choice(selected)
            if parent1 != parent2:
                child1, child2 = crossover(parent1, parent2)
                children.append(child1)
                children.append(child2)

        # Mutación
        mutated = []
        for i in range(int(N * m)):
            ind = random.choice(selected)
            mutated.append(mutate(ind, m))

        # Crear la nueva población
        population = selected + children + mutated

    # Resultados
    selected_items = [i for i in range(num_items) if best_solution[i] == 1]
    total_value = sum(v[i] for i in selected_items)
    total_weight = sum(w[i] for i in selected_items)
    
    return selected_items, total_value, total_weight


In [20]:
# Valores y pesos de los ítems
valores = [10, 12, 8, 5, 8, 5, 6, 7, 6, 12, 8, 8, 10, 9, 8, 3, 7, 8, 5, 6]
pesos = [6, 7, 7, 3, 5, 2, 4, 5, 3, 9, 8, 7, 8, 6, 5, 2, 3, 5, 4, 6]
capacidad = 50

# Llamada al algoritmo genético
items_seleccionados, valor_total, peso_total = knapsack_genetic(valores, pesos, capacidad)

print("Ítems seleccionados:", items_seleccionados)
print("Valor total:", valor_total)
print("Peso total:", peso_total)

KeyboardInterrupt: 